# Shuffling Methods

test cases

In [22]:
test_inputs = ['abc', 'def', 'egh', 'ijk', 'lmn', 'opq', 'rst', 'ade', 'des', 'asd']
test_targets = [1, 2, 1, 3, 2, 1, 2, 3, 4, 3]
expected_output = [
    ('abc', 1),
    ('egh', 1),
    ('opq', 1),
    ('def', 2),
    ('lmn', 2),
    ('rst', 2),
    ('ijk', 3),
    ('ade', 3),
    ('asd', 3),
    ('des', 4)
]

implementation

ideas for shuffle one class:
- sort the list by class, take as many pairs for the batches as possible
- build a dictionary with the classes and take batches from them

In [23]:
def shuffle_one_class(inputs, targets, batch_size=64):
    
    assert len(inputs) == len(targets), 'Inputs and targets do not have the same size'
    ### idea:
    # sort the input/target pairs, so that given a batch size the pairs can automatically be extracted
    # if one series is to small for a batch, fill it with the other missing ones
    # i.e. first create all fitting batches, then fill up missing ones
    new_idxs = [idx for idx, target in sorted(list(enumerate(targets)), key=lambda x: x[1])]
    sorted_inputs, sorted_targets = [inputs[idx] for idx in new_idxs], [targets[idx] for idx in new_idxs]
    
    # --> run over sorted inputs
    
    
    return sorted_inputs, sorted_targets
    
actual_output = list(zip(*shuffle_one_class(test_inputs, test_targets)))
assert actual_output == expected_output, 'Actual and expected outputs differ!'

In [4]:
def shuffle_all_classes(inputs, targets):
    pass

---

test on real data

In [ ]:
from utils.data_utils import DataLoader
dataloader = DataLoader(batch_sizes={'train': 64, 'test': 64}, shuffle={'train': False, 'test': False})
dataloader.download_cifar()
trainloader, testloader = dataloader.get_loaders()

for batch_idx, (inputs, targets) in enumerate(testloader):
    _inputs = inputs
    _targets = targets
    break